In [10]:
import pandas as pd

In [11]:
data = pd.read_csv("/content/synthetic_banking_conversations_2000.csv")

In [14]:
data.head(5)

,post_id,platform,bank_name,post_text,category,subcategory,sentiment_label,sentiment_score,date,user_id,...,language,country,is_verified,topic_tags,extracted_entities,urgency_flag,complaint_flag,resolved_flag,response_time_hours,source_url
0,P000001,Reddit,First National Bank,Account flagged for suspicious activity; suppo...,Security & Fraud,Unauthorized Transaction,Positive,0.398,2024-11-23,U81482,...,en,AU,True,security,First National Bank,False,False,NaN,NaN,https://example.com/post/1
1,P000002,Yelp,First National Bank,Session keeps timing out while I complete form...,Online Banking,UX,Negative,-0.243,2023-10-12,U10851,...,en,CA,False,rewards|loan,First National Bank,True,False,NaN,NaN,https://example.com/post/2
2,P000003,Reddit,SafeTrust Bank,"Found the ATM network reliable in my city, no ...",ATM Service,Network Down,Positive,0.256,2023-02-14,U70217,...,en,AU,True,support|ux,SafeTrust Bank,False,False,NaN,NaN,https://example.com/post/3
3,P000004,Yelp,SafeTrust Bank,Opening an account online was quick and straig...,Account Opening,Online Sign-up,Neutral,0.167,2025-06-06,U23238,...,en,AU,False,loan|ux,SafeTrust Bank,False,False,NaN,NaN,https://example.com/post/4
4,P000005,LinkedIn,Maple Savings,Partner deals are useful and save me money at ...,Rewards & Offers,Promotions,Positive,1.134,2024-06-30,U42087,...,en,AU,False,transfer|security|ux,Maple Savings,False,False,NaN,NaN,https://example.com/post/5


In [15]:
text_data = data[['post_id', 'platform', 'post_text']]

In [16]:
text_data.head(2)

,post_id,platform,post_text
0,P000001,Reddit,Account flagged for suspicious activity; suppo...
1,P000002,Yelp,Session keeps timing out while I complete form...


In [17]:
from transformers import pipeline

# Load a pretrained sentiment model
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [18]:
examples = [
    "Customer support was very helpful and resolved my issue within an hour.",
    "App login fails after the latest update, can't access my account.",
    "Great promotional offer for new credit cards this month!"
]

for text in examples:
    print(text, "→", sentiment_pipeline(text))


Customer support was very helpful and resolved my issue within an hour. → [{'label': 'POSITIVE', 'score': 0.9799594879150391}]
App login fails after the latest update, can't access my account. → [{'label': 'NEGATIVE', 'score': 0.9997230172157288}]
Great promotional offer for new credit cards this month! → [{'label': 'POSITIVE', 'score': 0.9989493489265442}]


In [19]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()  # shows progress bar

text_data["sentiment"] = text_data["post_text"].progress_apply(lambda x: sentiment_pipeline(x)[0]["label"])
text_data["sentiment_score"] = text_data["post_text"].progress_apply(lambda x: sentiment_pipeline(x)[0]["score"])


 63%|██████▎   | 1261/2000 [01:54<01:07, 10.98it/s]


KeyboardInterrupt: 

In [ ]:
text_data.head(3)

In [ ]:
text_data['sentiment'].value_counts()

not the most accurate sentiment (doesn't match the synthetic data)

In [ ]:
import sys
!{sys.executable} -m pip install transformers torch datasets --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "ProsusAI/finbert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

text = "Session keeps timing out while I complete forms online."
print(sentiment_pipeline(text))


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

text = "Session keeps timing out while I complete forms online."
print(sentiment_pipeline(text))


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
from tqdm import tqdm

# Load your dataset
hugging_face_data = data[['post_id', 'platform', 'post_text']]

model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [ ]:
def get_sentiment(text):
    try:
        result = sentiment_pipeline(text[:512])[0]  # limit long posts
        label = result["label"].lower()
        score = result["score"]
        if "neg" in label:
            signed_score = -(0.5 + 0.5 * score)
        elif "pos" in label:
            signed_score = (0.5 + 0.5 * score)
        else:
            signed_score = 0.0
        return pd.Series([label, signed_score])
    except Exception as e:
        return pd.Series(["error", 0.0])


In [ ]:
tqdm.pandas()
hugging_face_data[["pred_label", "pred_score"]] = hugging_face_data["post_text"].progress_apply(get_sentiment)


In [20]:
hugging_face_data.head(5)

NameError: name 'hugging_face_data' is not defined

In [34]:
absa_pipeline = pipeline(
    "text-classification",
    model="yangheng/deberta-v3-base-absa-v1.1",
    device_map="auto",
    top_k=None
)

example_review = "The mobile app is very convenient, but the credit card support is terrible and the loan process takes forever."

# for possible aspects we can use the categories that were in the sentiment data?
possible_aspects = ["mobile app", "credit card", "loan process", "customer service"]


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cpu


In [57]:
results = {}
for aspect in possible_aspects:
    result = absa_pipeline(example_review, text_pair=aspect)
    print(result)
    best = max(result[0], key=lambda x: x['score'])
    results[aspect] = best['label']

for aspect, sentiment in results.items():
    print(f"{aspect}: {sentiment}")

[[{'label': 'Positive', 'score': 0.9971936941146851}, {'label': 'Neutral', 'score': 0.0017095752991735935}, {'label': 'Negative', 'score': 0.001096732565201819}]]
[[{'label': 'Negative', 'score': 0.9966415166854858}, {'label': 'Positive', 'score': 0.0022131053265184164}, {'label': 'Neutral', 'score': 0.0011453808983787894}]]
[[{'label': 'Negative', 'score': 0.996661901473999}, {'label': 'Positive', 'score': 0.0021821404807269573}, {'label': 'Neutral', 'score': 0.001156090060248971}]]
[[{'label': 'Negative', 'score': 0.9805827736854553}, {'label': 'Positive', 'score': 0.01536619197577238}, {'label': 'Neutral', 'score': 0.004051117226481438}]]
mobile app: Positive
credit card: Negative
loan process: Negative
customer service: Negative


In [63]:
# testing on a portion of the synthetic data now
subframe = data.head(10)
reviews = subframe["post_text"].tolist()
categories = data["category"].unique().tolist()

print(reviews)
categories

['Account flagged for suspicious activity; support helped secure it. #customerservice', 'Session keeps timing out while I complete forms online.', 'Found the ATM network reliable in my city, no issues.', 'Opening an account online was quick and straightforward. #banking', 'Partner deals are useful and save me money at checkout. @ SafeTrust Bank', 'Navigation is intuitive and helpful for finding statements.', 'Received phishing email pretending to be the bank, be careful.', 'Found the ATM network reliable in my city, no issues. (shared experience)', 'I was charged an unexpected international transfer fee. Not cool. #customerservice', 'Exchange rate was competitive and transfer arrived on time. - happened in Toronto']


['Security & Fraud',
 'Online Banking',
 'ATM Service',
 'Account Opening',
 'Rewards & Offers',
 'Website Experience',
 'Fees & Charges',
 'International Transfers',
 'Interest Rates',
 'App Performance',
 'Customer Service',
 'Loan Process',
 'Notifications',
 'Credit Card',
 'Branch Experience']

In [61]:
results = {}

for review in reviews:
    for category in categories:
        result = absa_pipeline(review, text_pair=category)
        best = max(result[0], key=lambda x: x['score'])
        results[category] = best['label']

for aspect, sentiment in results.items():
    print(f"{aspect}: {sentiment}")

Security & Fraud: Positive
Online Banking: Neutral
ATM Service: Positive
Account Opening: Negative
Rewards & Offers: Neutral
Website Experience: Positive
Fees & Charges: Neutral
International Transfers: Neutral
Interest Rates: Negative
App Performance: Positive
Customer Service: Neutral
Loan Process: Neutral
Notifications: Positive
Credit Card: Neutral
Branch Experience: Positive


In [64]:
from collections import defaultdict, Counter

sentiment_counts = defaultdict(Counter)

for review in reviews:
    for category in categories:
        result = absa_pipeline(review, text_pair=category)
        best = max(result[0], key=lambda x: x['score'])
        sentiment_counts[category][best['label']] += 1

for category, counts in sentiment_counts.items():
    print(f"{category}: {dict(counts)}")


Security & Fraud: {'Positive': 2, 'Negative': 8}
Online Banking: {'Neutral': 7, 'Positive': 2, 'Negative': 1}
ATM Service: {'Positive': 7, 'Negative': 2, 'Neutral': 1}
Account Opening: {'Negative': 3, 'Neutral': 4, 'Positive': 3}
Rewards & Offers: {'Neutral': 4, 'Negative': 2, 'Positive': 4}
Website Experience: {'Positive': 5, 'Negative': 2, 'Neutral': 3}
Fees & Charges: {'Neutral': 8, 'Negative': 2}
International Transfers: {'Neutral': 6, 'Negative': 2, 'Positive': 2}
Interest Rates: {'Negative': 4, 'Neutral': 2, 'Positive': 4}
App Performance: {'Positive': 7, 'Negative': 2, 'Neutral': 1}
Customer Service: {'Neutral': 1, 'Negative': 3, 'Positive': 6}
Loan Process: {'Neutral': 6, 'Negative': 2, 'Positive': 2}
Notifications: {'Positive': 3, 'Negative': 2, 'Neutral': 5}
Credit Card: {'Neutral': 7, 'Negative': 2, 'Positive': 1}
Branch Experience: {'Positive': 4, 'Negative': 2, 'Neutral': 4}
